In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [2]:
import os
import sys
import logging
import argparse
os.environ["DATASET"] = "../datasets"

pth = '/'.join(sys.path[0].split('/')[:-1])
sys.path.insert(0, pth)

from pprint import pprint
import numpy as np
np.random.seed(1)

home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
print(home_dir)

import warnings
warnings.filterwarnings(action='ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from hdecoder.BaseModel import BaseModel
from hdecoder import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files, load_config_dict_to_opt
from utils.misc import MetricLogger

/home/djjin/Mygit/X-Decoder


Invalid MIT-MAGIC-COOKIE-1 key

In [6]:
from utils.arguments import load_vcoco_opt_command, load_vcoco_parser

cmdline_args = load_vcoco_parser()
cmdline_args.conf_files = [os.path.join(home_dir, "configs/hdecoder/vcoco.yaml")]

model_path = '../data/output/test/00205200/default/raw_model_states.pt'
cmdline_args.overrides = ['DONT_LOAD_MODEL', 'false', 'PYLEARN_MODEL', model_path] 

opt = load_vcoco_opt_command(cmdline_args)
opt = init_distributed(opt)

In [7]:
print(opt['base_path'])
print(opt["RESUME"])

../
False


In [8]:
from trainer import HDecoder_Trainer as Trainer
if cmdline_args.user_dir:
    absolute_user_dir = os.path.abspath(cmdline_args.user_dir)
    opt['base_path'] = absolute_user_dir
trainer = Trainer(opt)
trainer.eval()

INFO:trainer.distributed_trainer:Setting SAVE_DIR as data/output/test
INFO:trainer.distributed_trainer:Using CUDA
INFO:trainer.distributed_trainer:Save config file to data/output/test/conf_copy.yaml
INFO:trainer.distributed_trainer:Base learning rate: 0.0001
INFO:trainer.distributed_trainer:Number of GPUs: 1
INFO:trainer.distributed_trainer:Gradient accumulation steps: 1
INFO:trainer.default_trainer:Imported base_dir at base_path ../
INFO:trainer.default_trainer:Pipeline for training: HDecoderPipeline
INFO:trainer.default_trainer:-----------------------------------------------
INFO:trainer.default_trainer:Evaluating model ... 
INFO:base_dir.pipeline.HDecoderPipeline:CDNHOI(
  (backbone): D2FocalNet(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (bloc

Evaluation start ...


INFO:detectron2.data.common:Serializing 4946 elements to byte tensors and concatenating them all ...
INFO:detectron2.data.common:Serialized dataset takes 3.69 MiB
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 11/2473. Dataloading: 0.0247 s/iter. Inference: 0.4678 s/iter. Eval: 0.0005 s/iter. Total: 0.4930 s/iter. ETA=0:20:13
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 22/2473. Dataloading: 0.0274 s/iter. Inference: 0.4684 s/iter. Eval: 0.0004 s/iter. Total: 0.4963 s/iter. ETA=0:20:16
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 32/2473. Dataloading: 0.0295 s/iter. Inference: 0.4692 s/iter. Eval: 0.0004 s/iter. Total: 0.4992 s/iter. ETA=0:20:18
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 42/2473. Dataloading: 0.0300 s/iter. Inference: 0.4706 s/iter. Eval: 0.0004 s/iter. Total: 0.5011 s/iter. ETA=0:20:18
INFO:base_dir.pipeline.HDecoderPipeline:Inference done 52/2473. Dataloading: 0.0296 s/iter. Inference: 0.4716 s/iter. Eval: 0.0004 s/iter. Total:

------------------------------------------------------------
               hold_obj: #GTs = 3608, AP = 0.3483
                  stand: #GTs = 4118, AP = 0.7491
              sit_instr: #GTs = 1916, AP = 0.3896
             ride_instr: #GTs = 0556, AP = 0.4844
                   walk: #GTs = 0597, AP = 0.4764
               look_obj: #GTs = 3347, AP = 0.3360
              hit_instr: #GTs = 0349, AP = 0.5331
                hit_obj: #GTs = 0349, AP = 0.1892
                eat_obj: #GTs = 0521, AP = 0.4527
              eat_instr: #GTs = 0521, AP = 0.6312
             jump_instr: #GTs = 0635, AP = 0.6612
              lay_instr: #GTs = 0387, AP = 0.4775
    talk_on_phone_instr: #GTs = 0285, AP = 0.2361
              carry_obj: #GTs = 0472, AP = 0.2927
              throw_obj: #GTs = 0244, AP = 0.2757
              catch_obj: #GTs = 0246, AP = 0.3038
              cut_instr: #GTs = 0269, AP = 0.1759
                cut_obj: #GTs = 0269, AP = 0.5231
                    run: #GTs = 0687, A

INFO:trainer.default_trainer:{'vcoco_val/vcoco': {'AP_hold_obj': 0.3483403578158819, 'AP_stand': 0.749058574769091, 'AP_sit_instr': 0.3896438485535703, 'AP_ride_instr': 0.4843836571324491, 'AP_walk': 0.4763679007195397, 'AP_look_obj': 0.33598909310510644, 'AP_hit_instr': 0.5331438806295269, 'AP_hit_obj': 0.18923280115333918, 'AP_eat_obj': 0.4526988889337091, 'AP_eat_instr': 0.6312347122923342, 'AP_jump_instr': 0.6612438295423342, 'AP_lay_instr': 0.47753235134313365, 'AP_talk_on_phone_instr': 0.23609793073657673, 'AP_carry_obj': 0.29266659048476573, 'AP_throw_obj': 0.2756762000676301, 'AP_catch_obj': 0.30381678458873074, 'AP_cut_instr': 0.1758579932948957, 'AP_cut_obj': 0.5231306868206864, 'AP_run': 0.6767946793527677, 'AP_work_on_computer_instr': 0.5922039848335071, 'AP_ski_instr': 0.31092090684402635, 'AP_surf_instr': 0.6234337146248906, 'AP_skateboard_instr': 0.6589723199908812, 'AP_smile': 0.7306451894241708, 'AP_drink_instr': 0.2989242329683353, 'AP_kick_obj': 0.690995501189503, 'A

{'vcoco_val/vcoco': {'AP_hold_obj': 0.3483403578158819,
  'AP_stand': 0.749058574769091,
  'AP_sit_instr': 0.3896438485535703,
  'AP_ride_instr': 0.4843836571324491,
  'AP_walk': 0.4763679007195397,
  'AP_look_obj': 0.33598909310510644,
  'AP_hit_instr': 0.5331438806295269,
  'AP_hit_obj': 0.18923280115333918,
  'AP_eat_obj': 0.4526988889337091,
  'AP_eat_instr': 0.6312347122923342,
  'AP_jump_instr': 0.6612438295423342,
  'AP_lay_instr': 0.47753235134313365,
  'AP_talk_on_phone_instr': 0.23609793073657673,
  'AP_carry_obj': 0.29266659048476573,
  'AP_throw_obj': 0.2756762000676301,
  'AP_catch_obj': 0.30381678458873074,
  'AP_cut_instr': 0.1758579932948957,
  'AP_cut_obj': 0.5231306868206864,
  'AP_run': 0.6767946793527677,
  'AP_work_on_computer_instr': 0.5922039848335071,
  'AP_ski_instr': 0.31092090684402635,
  'AP_surf_instr': 0.6234337146248906,
  'AP_skateboard_instr': 0.6589723199908812,
  'AP_smile': 0.7306451894241708,
  'AP_drink_instr': 0.2989242329683353,
  'AP_kick_obj': 

In [ ]:
@torch.no_grad()
def generate(model, post_processor, data_loader, device, verb_classes, missing_category_id):
    model.eval()

    metric_logger = MetricLogger(delimiter="  ")
    header = 'Generate:'

    detections = []
    for samples, targets in metric_logger.log_every(data_loader, 10, header):
        samples = samples.to(device)

        outputs = model(samples)
        orig_target_sizes = torch.stack([t["orig_size"] for t in targets], dim=0)

        results = post_processor(outputs, orig_target_sizes)

        for img_results, img_targets in zip(results, targets):
            for hoi in img_results['hoi_prediction']:
                detection = {
                    'image_id': img_targets['img_id'],
                    'person_box': img_results['predictions'][hoi['subject_id']]['bbox'].tolist()
                }
                if img_results['predictions'][hoi['object_id']]['category_id'] == missing_category_id:
                    object_box = [np.nan, np.nan, np.nan, np.nan]
                else:
                    object_box = img_results['predictions'][hoi['object_id']]['bbox'].tolist()
                cut_agent = 0
                hit_agent = 0
                eat_agent = 0
                for idx, score in zip(hoi['category_id'], hoi['score']):
                    verb_class = verb_classes[idx]
                    score = score.item()
                    if len(verb_class.split('_')) == 1:
                        detection['{}_agent'.format(verb_class)] = score
                    elif 'cut_' in verb_class:
                        detection[verb_class] = object_box + [score]
                        cut_agent = score if score > cut_agent else cut_agent
                    elif 'hit_' in verb_class:
                        detection[verb_class] = object_box + [score]
                        hit_agent = score if score > hit_agent else hit_agent
                    elif 'eat_' in verb_class:
                        detection[verb_class] = object_box + [score]
                        eat_agent = score if score > eat_agent else eat_agent
                    else:
                        detection[verb_class] = object_box + [score]
                        detection['{}_agent'.format(
                            verb_class.replace('_obj', '').replace('_instr', ''))] = score
                detection['cut_agent'] = cut_agent
                detection['hit_agent'] = hit_agent
                detection['eat_agent'] = eat_agent
                detections.append(detection)

    return detections
